In [ ]:
%pylab inline
from __future__ import print_function, division, absolute_import
from statsmodels.kernel_methods import kde, kernels, bandwidths, kde_methods
from scipy import stats, integrate

## Defining the target distribution

In [ ]:
dist1 = stats.norm(loc=1.2, scale=0.4)
dist2 = stats.norm(loc=-3, scale=2)
N = int(1e4)
d = r_[dist1.rvs(N), dist2.rvs(N)]
k = kde.KDE(d, method=kde_methods.Cyclic())
est = k.fit()
xs = r_[-12:6:2048j]
def pdf(x):
    return (dist1.pdf(x) + dist2.pdf(x))/2

### Estimation with Scott's rule of thumb:

In [ ]:
f = figure()
gr, gr_pdf = est.grid()
plot(gr.full(), gr_pdf)
plot(xs, pdf(xs), 'r--')
title("Scott's rule of thumb for bandwidth estimation")

## Estimation with cross-validation

**Note:** We will use the folding method with 10 folds and grid-based approximation

In [ ]:
%%time
k1 = k.copy()
k1.bandwidth = bandwidths.lsq_crossvalidation(imse_args=dict(folding=10, use_grid=True))
est1 = k1.fit()

In [ ]:
est1.bandwidth

In [ ]:
est.bandwidth

In [ ]:
f = figure()
xs1, pdf1 = est1.grid()
plot(xs1.full(), pdf1)
plot(xs, pdf(xs), 'r--')
ylim(ymin=0)
title('Cross-validation for bandwidth estimation')

The result is still much much better

### Full Leave-One-Out method for the cross-validation

In [ ]:
%%time
k2 = k.copy()
k2.bandwidth = bandwidths.lsq_crossvalidation()
est2 = k2.fit()

In [ ]:
est2.bandwidth

### Using Randomized leave-one-out method for the cross-validation

In [ ]:
%%time
k3 = k.copy()
k3.bandwidth = bandwidths.lsq_crossvalidation(imse_args=dict(sampling=1000))
est3 = k3.fit()

In [ ]:
est3.bandwidth

## Testing IMSE speed

In [ ]:
imse_sample = bandwidths.ContinuousIMSE(k, sampling=3000)

In [ ]:
%time imse_sample(0.3)

In [ ]:
imse_loo = bandwidths.ContinuousIMSE(k)

In [ ]:
%time imse_loo(0.3)

In [ ]:
imse_fold = bandwidths.ContinuousIMSE(k, folding=10, use_grid=True)

In [ ]:
%time imse_fold(0.3)

In [ ]:
imse_sample2 = bandwidths.ContinuousIMSE(k, sampling=3000, use_grid=True)
%time imse_sample2(0.3)